### Info
based on notebook4

compare qkeras and qonnx predictions (without using hls4ml)\
strange thing on github: pred_qonnx[0]: [[4.4136963]]; then: 4.4375

### Load packages and the model

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copy weights

In [ ]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### QONNX model

In [1]:
#Quick setup (load qonnx model)
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits

import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper

qonnx_model = ModelWrapper("mod_model.onnx")

2025-09-29 14:36:49.266664: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-29 14:36:49.896540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-29 14:36:49.896603: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-29 14:36:49.901200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-29 14:36:50.346208: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-29 14:36:50.349385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
#from qonnx.transformation.infer_shapes import InferShapes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)

In [25]:
node_names = []
for node in qonnx_model.graph.node:
    node_names.append(node.name)

print(node_names)

['Quant_0', 'Quant_1', 'Quant_2', 'Quant_3', 'Quant_4', 'Quant_5', 'Quant_6', 'Quant_7', 'Quant_8', 'Quant_9', 'Quant_10', 'Quant_11', 'Quant_12', 'Quant_13', 'Quant_14', 'Quant_15', 'MatMul_0', 'Add_0', 'Quant_16', 'Relu_0', 'Quant_17', 'MatMul_1', 'Add_1', 'Quant_18', 'Relu_1', 'Quant_19', 'MatMul_2', 'Add_2', 'Quant_20', 'Relu_2', 'Quant_21', 'MatMul_3', 'Add_3', 'Quant_22', 'Relu_3', 'Quant_23', 'MatMul_4', 'Add_4', 'Quant_24', 'Relu_4', 'Quant_25', 'MatMul_5', 'Add_5', 'Quant_26', 'Relu_5', 'Quant_27', 'MatMul_6', 'Add_6', 'Quant_28', 'Relu_6', 'Quant_29', 'MatMul_7', 'Add_7', 'Sub_0', 'Unsqueeze_0', 'Unsqueeze_1', 'MatMul_8', 'Squeeze_0', 'Identity_0', 'Identity_1']


### Check the input parameters of the Quant nodes (adjust the inputs and attributes)
0.125 = (0.001)_2 \
0.0625 = (0.0001)_2 \
0.0009765625 = (0.0000000001)_2

In [ ]:
qnodes = list(filter(lambda x: x.op_type == "Quant", qonnx_model.graph.node))

# qnode consumers and param inputs
for qnode in qnodes[16:]:
    print("Quant node name: ",qnode.name)
    print("Consumer name: ",qonnx_model.find_consumer(qnode.output[0]).name)
    for inp in qnode.input[1:]:
        print(inp, qonnx_model.get_initializer(inp))
    print("--------------------")
# Only the Quant_19 node has two consumers (value after 'MatMul_2', later also 'Sub_0')

In [ ]:
qonnx_model.set_initializer("Quant_16_param0",np.array(0.0009765625))

In [ ]:
# qnode attributes
for qnode in qnodes[16:17]:
    print("Quant node name: ",qnode.name)
    print(qnode.attribute)
    print("---------------")

In [ ]:
from qonnx.custom_op.base import CustomOp
from qonnx.custom_op.registry import getCustomOp

node_op = getCustomOp(qonnx_model.get_finn_nodes()[16])

In [ ]:
node_op.set_nodeattr("signed",1)

### Conclusions
In hls input_1 (global input) should have precision fixed<14,7> (now fixed<18,8>)\
Relu should take input with precision like the accumulator: fixed<18,8>\
Should check the "signed" attribute of Quant nodes\
missing quantizers before 'Sub_0' and later on

Quant node is not present between each add and relu within the Dense layers.\
These quant nodes should correspond to accumulators (?) \
Check inputs of the Subtract node \
Strange "Linear" layers in the hls_config

#from qonnx.transformation.general import ApplyConfig

### Add missing Quant nodes

In [23]:
from qonnx.transformation.quantize_graph import QuantizeGraph

qnode_map = {"name": {"Relu_3_4": [(("output",0), (scale_,zeropoint_,bitwidth_,narrow_,signed_,rounding_mode_))]}}
qnode_map

{'name': {'Relu_3_4': [(('output', 0), (0.0009765625, 0, 18, 0, 0, 'ROUND'))]}}

In [22]:
scale_ = 0.0009765625
zeropoint_ = 0
bitwidth_ = 18
narrow_ = 0
signed_ = 0
rounding_mode_ = "ROUND"

In [24]:
quantize_graph = QuantizeGraph(qnode_map)
(qonnx_model,_) = quantize_graph.apply(qonnx_model)

### Add missing standard nodes

In [ ]:
from onnx.helper import make_tensor_value_info
from onnx import TensorProto

relu_3_4_out_tens_vinfo = make_tensor_value_info(
    "Relu_3_4_out_0",
    TensorProto.FLOAT,
    shape_,
)

qonnx_model.graph.value_info.append(relu_3_4_out_tens_vinfo)

In [20]:
print(qonnx_model.graph.node[25:])

[input: "Relu_1_out0"
input: "Quant_19_param0"
input: "Quant_19_param1"
input: "Quant_19_param2"
output: "Quant_19_out0"
name: "Quant_19"
op_type: "Quant"
attribute {
  name: "narrow"
  i: 0
  type: INT
}
attribute {
  name: "rounding_mode"
  s: "ROUND"
  type: STRING
}
attribute {
  name: "signed"
  i: 0
  type: INT
}
domain: "qonnx.custom_op.general"
, input: "Quant_19_out0"
input: "Quant_10_out0"
output: "MatMul_2_out0"
name: "MatMul_2"
op_type: "MatMul"
, input: "MatMul_2_out0"
input: "Quant_11_out0"
output: "Add_2_out0"
name: "Add_2"
op_type: "Add"
, input: "Add_2_out0"
input: "Quant_20_param0"
input: "Quant_20_param1"
input: "Quant_20_param2"
output: "Quant_20_out0"
name: "Quant_20"
op_type: "Quant"
attribute {
  name: "narrow"
  i: 0
  type: INT
}
attribute {
  name: "rounding_mode"
  s: "ROUND"
  type: STRING
}
attribute {
  name: "signed"
  i: 0
  type: INT
}
domain: "qonnx.custom_op.general"
, input: "Quant_20_out0"
output: "Relu_2_out0"
name: "Relu_2"
op_type: "Relu"
, input

In [19]:
qonnx_model.get_node_from_name("MatMul_5").input

['Relu_3_4_out0', 'Quant_6_out0']

In [11]:
# Relu does not change tensor shape
shape_ = qonnx_model.get_tensor_shape("Quant_24_out0")

In [ ]:
qonnx_model.get_node_from_name("Relu_1")

In [ ]:
qonnx_model.get_node_index(qonnx_model.get_node_from_name("Relu_1"))

In [ ]:
type(qonnx_model.get_node_from_name("Quant_24").output[0])

In [ ]:
from onnx.helper import make_node

relu_node = make_node(
    "Relu",
    inputs=qonnx_model.get_node_from_name("Quant_24").output,
    outputs=["Relu_3_4_out0"],
    name="Relu_3_4"
)

In [ ]:
qonnx_model.graph.node.insert(qonnx_model.get_node_index(qonnx_model.get_node_from_name("Quant_24"))+1,relu_node)

In [ ]:
node_index = qonnx_model.get_node_index(qonnx_model.get_node_from_name("MatMul_5"))
input_list = qonnx_model.graph.node[node_index].__getattribute__("input")
print(input_list)

In [26]:
qonnx_model.save("mod_model_2.onnx")